In [29]:
#set openai key in .env.local
import os
import pandas as pd
import openai
import numpy as np
#from openai.embeddings_utils import get_embedding

In [30]:
def estimate_price(n):
    num_tokens = 100 * n
    # per thousend tokens 0.0001 $
    price_per_thousend = 0.0001
    price = round(num_tokens/1000 * price_per_thousend, 5)

    response = input(f"Estimated price {price} $\nProceed? (y/n): ")
    if response.lower() == 'y':
        print("Price accepted...")
        # Continue with your notebook code here

    elif response.lower() == 'n':
        print("Price declined, interrupting...")
        raise KeyboardInterrupt  # Raises a KeyboardInterrupt to stop execution

    else:
        print("Invalid input. interrupting...")
        raise KeyboardInterrupt
        # Prompt the user again or take appropriate action


In [31]:
def set_environment_variables_from_file(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()

    for line in lines:
        line = line.strip()
        if line and not line.startswith('#'):  # Ignore empty lines and comments
            key, value = line.split('=', 1)
            os.environ[key.strip()] = value.strip()
file_path = '../../.env.local'
set_environment_variables_from_file(file_path)
openai.api_key = os.getenv('OPENAI_KEY')

In [32]:
# embedding model parameters
embedding_model = "text-embedding-ada-002"
embedding_encoding = "cl100k_base"  # this the encoding for text-embedding-ada-002
max_tokens = 8000  # the maximum for text-embedding-ada-002 is 8191

In [33]:
basepath = '../../datasets/preprocessed_datasets/gabor/'
product_data = pd.read_pickle(basepath +'no_product_variants.pkl')
product_data = product_data.reset_index(drop=True)
product_data = product_data.sample(n=2000, random_state=2).reset_index(drop=True)
display(product_data)

,product__id,productNumber,productName,productSizeEu,productColorName,articleNumber,brand__id,mainCategory__id,gender,originCountry,...,shoeStyle,productType_y,heelHeightGroup,sizeEu,sizeUk,sizeIndex,shaftLength__value,sole,isTransferee,isSuccessor
0,3696208,4062862013129,Sneaker low Materialmix Leder/Textil grau,42.0,grau,1007.10.03,3692257,3692261,m,CN,...,Sneaker,Herrenschuhe,3 cm - 5 cm,42.0,8.0,7.0,0.0,Gummi-EVA,0.0,0.0
1,10310967,4065171998385,Slingpumps Rauleder Lila,38.5,Lila,21.521.13,6590677,315565,w,PT,...,Pumps,Damenschuhe,3 cm - 5 cm,38.5,5.5,9.0,0.0,TPU,0.0,0.0
2,3479727,4062862475613,Schnürstiefelette Rauleder gelb,42.0,gelb,52.076.30,6590678,315587,w,SK,...,Stiefeletten,Damenschuhe,bis 3 cm,42.0,8.0,14.0,14.0,TR,0.0,0.0
3,7593992,4064032904039,Plateau Sandale Materialmix Leder braun,35.0,braun,82.736.52,6590678,315568,w,VN,...,Sandalen,Damenschuhe,5 cm - 8 cm,35.0,2.5,3.0,0.0,Gummi,1.0,1.0
4,8504015,4065171432780,Enkellaarsjes op plateau Suède grijs,44.0,grijs,94.780.13,6590677,315595,w,SK,...,Stiefeletten,Damenschuhe,5 cm - 8 cm,44.0,9.5,17.0,14.0,TR,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,7595163,4064032261774,Riemchensandale Glattleder braun,44.0,braun,82.773.51,6590678,315564,w,PT,...,Sandalen,Damenschuhe,3 cm - 5 cm,44.0,9.5,17.0,0.0,TR,0.0,0.0
1996,4770623,4062862808909,Plateau Sandale Rauleder,42.0,,64.614.17,6590677,315563,w,SK,...,Sandalen,Damenschuhe,5 cm - 8 cm,42.0,8.0,14.0,0.0,PU-TPU,0.0,1.0
1997,9218621,4065171587046,Plüschhausschuh Textil rosa,38.0,rosa,93.060.10,6590677,8487167,w,PT,...,Hausschuhe,Damenschuhe,bis 3 cm,38.0,5.0,6.0,0.0,EVA,0.0,0.0
1998,8666377,4065171660602,Lage sneaker Glad leer bruin,35.0,bruin,93.334.24,6590677,315567,w,SK,...,Sneaker,Damenschuhe,3 cm - 5 cm,35.0,2.5,3.0,0.0,TR,1.0,1.0


In [34]:
def combine_columns(row):
    return (
            "ID: " + str(row.product__id) + "; " +
            "product_number: " + str(row.productNumber) + "; " +
            "product_name: " + row.productName + "; " +
            # "size: " + str(row.productSizeEu) + "; " +
            "color: " + str(row.productColorName) + "; " +
            "article number: " + str(row.articleNumber) + "; " +
            "brand: " + str(row.brand__id) + "; " +
            "main category: " + str(row.mainCategory__id) + "; " +
            "gender: " + str(row.gender) + "; " +
            "line: " + str(row.line) + "; " +
            "price: " + str(row.b2cPrice) + "; " +
            "material: " + str(row.material) + "; " +
            "material PM: " + str(row.materialPm) + "; "
    )

def get_embedding(text, model="text-embedding-ada-002"):
    text = text.replace("\n", " ")
    return openai.Embedding.create(input = [text], model=model)['data'][0]['embedding']

def get_embeddings(texts, model="text-embedding-ada-002"):
    return openai.Embedding.create(input=texts, model=model).data

def get_list_from_embeddings(embedding_result):
    list = []
    for entry in embedding_result:
        list.append(entry.embedding)
    return list

In [35]:
product_embeddings_path = "openai/product_embeddings.pkl"
recreate_embeddings = True
if not os.path.exists(basepath + product_embeddings_path) or recreate_embeddings:
    product_data_embeddings = product_data[['product__id']].copy()
    product_data_embeddings["combined"] = product_data.apply(lambda row: combine_columns(row), axis=1)
    # print(product_data_embeddings.combined.tolist())
    estimate_price(product_data_embeddings.shape[0])
    # embed each row separately
    # product_data_embeddings['embedding'] = product_data_embeddings.combined.apply(lambda x: get_embedding(x, model='text-embedding-ada-002'))
    # batch embed
    product_data_embeddings['embedding'] = get_list_from_embeddings(get_embeddings(product_data_embeddings.combined.tolist()))
    product_data_embeddings.to_pickle(basepath + product_embeddings_path)
else:
    product_data_embeddings = pd.read_pickle(basepath + product_embeddings_path)
display(product_data_embeddings)

Price accepted...


,product__id,combined,embedding
0,3696208,ID: 3696208; product_number: 4062862013129; pr...,"[0.013865606859326363, 0.0059647392481565475, ..."
1,10310967,ID: 10310967; product_number: 4065171998385; p...,"[0.008594418875873089, 0.018468571826815605, -..."
2,3479727,ID: 3479727; product_number: 4062862475613; pr...,"[0.0033649103716015816, -0.0010019299807026982..."
3,7593992,ID: 7593992; product_number: 4064032904039; pr...,"[0.008496461436152458, -0.00178595632314682, -..."
4,8504015,ID: 8504015; product_number: 4065171432780; pr...,"[0.01740363799035549, -0.01851450838148594, -0..."
...,...,...,...
1995,7595163,ID: 7595163; product_number: 4064032261774; pr...,"[0.006299823988229036, 0.009966087527573109, -..."
1996,4770623,ID: 4770623; product_number: 4062862808909; pr...,"[0.006249087397009134, -0.0014839477371424437,..."
1997,9218621,ID: 9218621; product_number: 4065171587046; pr...,"[0.006515574641525745, 0.005344858393073082, -..."
1998,8666377,ID: 8666377; product_number: 4065171660602; pr...,"[-0.0006240483489818871, -0.001225750078447163..."


In [36]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_n_nearest_neighbors(query_embedding, neighbors_embedding, n = 5):
    similarities = [cosine_similarity(query_embedding, neighbor) for neighbor in neighbors_embedding]
    indices = np.flip(np.argsort(similarities)[-n:])
    scores = [similarities[i] for i in indices]
    # Get the n neighbors with the highest similarity values
    n_neighbors = [neighbors_embedding[i] for i in indices]

    return indices, scores


indices, scores = get_n_nearest_neighbors(product_data_embeddings.embedding[0], product_data_embeddings.embedding)
display(indices, scores)

array([   0, 1450, 1027,   11,  762])

[1.0,
 0.9885263210629753,
 0.9866243723324932,
 0.9831100128526201,
 0.9819263753691818]

In [37]:
product_data_embeddings.iloc[indices].combined

0       ID: 3696208; product_number: 4062862013129; pr...
1450    ID: 7594042; product_number: 4064032158098; pr...
1027    ID: 3845441; product_number: 4060666975391; pr...
11      ID: 3696172; product_number: 4062862012818; pr...
762     ID: 3695399; product_number: 4062862138150; pr...
Name: combined, dtype: object